In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from keras.optimizers import RMSprop, Adam
from utils.dataset import DataSet, LABELS
from models.fasttext import FastText
from models.sentiment_base import SCompositeModel as CompositeFastText
from env import *

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs', 'simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

SAVED_PATH = os.path.join(SAVED_MODELS_PATH, 'fasttext')
if not os.path.exists(SAVED_PATH):
    os.makedirs(SAVED_PATH)

MODEL_PRE = 'fasttext_'

LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 64
FACTOR = 0.2

EMBEDDING_PKL = os.path.join(WORD_SEQS_PATH, 'wem.pkl')
RESULT_CSV = os.path.join(RESULTS_PATH, 'fasttext.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)
    
with open(EMBEDDING_PKL, 'rb') as f:
    embedding = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

In [ ]:
y_cols = LABELS

seq = 'words_seq'

train_x = np.array(list(train_with_seq[seq]))
train_y = train_with_seq[y_cols]
val_x = np.array(list(val_with_seq[seq]))
val_y = val_with_seq[y_cols]
test_x = np.array(list(test_with_seq[seq]))

comps = [(FastText, {"max_len":train_x.shape[1], 
                    'embedding': embedding.copy(),
                    'mask_zero': False})] * len(y_cols)
# comps = [(LinXiSingleModel, {"maxlen":train_x.shape[1]})] * len(y_cols)
comp_model = CompositeFastText(comps)
comp_model.fit(train_x, train_y, val = (val_x, val_y), y_cols = y_cols, seq = 'word_seq', saved_path = SAVED_PATH,
               model_pre = MODEL_PRE, lr = LR, epochs = EPOCHS, batch_size = BATCH_SIZE, optimizer = RMSprop,
               factor = FACTOR)

Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 12s 117us/step - loss: 0.6087 - acc: 0.7750 - val_loss: 0.5314 - val_acc: 0.7860

Epoch 00001: _f1_monitor improved from -inf to 0.22670, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_0.h5
Epoch 2/100
  1472/105000 [..............................] - ETA: 8s - loss: 0.5250 - acc: 0.7948

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


105000/105000 [==============================] - 11s 104us/step - loss: 0.4841 - acc: 0.8153 - val_loss: 0.4259 - val_acc: 0.8549

Epoch 00002: _f1_monitor improved from 0.22670 to 0.37280, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_0.h5
Epoch 3/100
105000/105000 [==============================] - 10s 98us/step - loss: 0.3933 - acc: 0.8680 - val_loss: 0.3652 - val_acc: 0.8819

Epoch 00003: _f1_monitor improved from 0.37280 to 0.40555, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_0.h5
Epoch 4/100
105000/105000 [==============================] - 10s 98us/step - loss: 0.3450 - acc: 0.8917 - val_loss: 0.3376 - val_acc: 0.8960

Epoch 00004: _f1_monitor improved from 0.40555 to 0.42110, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_0.h5
Epoch 5/100
105000/105000 [==============================] - 10s 98us/step - loss: 0.3221 - acc:

105000/105000 [==============================] - 10s 98us/step - loss: 0.4030 - acc: 0.8529 - val_loss: 0.4341 - val_acc: 0.8428

Epoch 00009: _f1_monitor did not improve from 0.35269
Epoch 10/100
105000/105000 [==============================] - 10s 100us/step - loss: 0.3996 - acc: 0.8543 - val_loss: 0.4332 - val_acc: 0.8441

Epoch 00010: _f1_monitor did not improve from 0.35269
Epoch 11/100
105000/105000 [==============================] - 10s 98us/step - loss: 0.3959 - acc: 0.8558 - val_loss: 0.4316 - val_acc: 0.8440

Epoch 00011: _f1_monitor did not improve from 0.35269
Epoch 12/100
105000/105000 [==============================] - 10s 100us/step - loss: 0.3930 - acc: 0.8574 - val_loss: 0.4360 - val_acc: 0.8429

Epoch 00012: _f1_monitor improved from 0.35269 to 0.35869, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_1.h5
Epoch 13/100
105000/105000 [==============================] - 11s 103us/step - loss: 0.3902 - acc: 0.8579 - val_lo

105000/105000 [==============================] - 11s 106us/step - loss: 0.4369 - acc: 0.8582 - val_loss: 0.4533 - val_acc: 0.8549

Epoch 00006: _f1_monitor improved from 0.42117 to 0.45322, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_2.h5
Epoch 7/100
105000/105000 [==============================] - 11s 101us/step - loss: 0.4159 - acc: 0.8667 - val_loss: 0.4430 - val_acc: 0.8601

Epoch 00007: _f1_monitor improved from 0.45322 to 0.48125, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_2.h5
Epoch 8/100
105000/105000 [==============================] - 11s 103us/step - loss: 0.4020 - acc: 0.8728 - val_loss: 0.4374 - val_acc: 0.8659

Epoch 00008: _f1_monitor improved from 0.48125 to 0.49461, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_2.h5
Epoch 9/100
105000/105000 [==============================] - 11s 101us/step - loss: 0.3930 - a

...

105000/105000 [==============================] - 11s 108us/step - loss: 0.4122 - acc: 0.8759 - val_loss: 0.4279 - val_acc: 0.8761

Epoch 00005: _f1_monitor improved from 0.40214 to 0.44312, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_6.h5
Epoch 6/100
105000/105000 [==============================] - 11s 104us/step - loss: 0.3942 - acc: 0.8822 - val_loss: 0.4135 - val_acc: 0.8771

Epoch 00006: _f1_monitor did not improve from 0.44312
Epoch 7/100
105000/105000 [==============================] - 11s 101us/step - loss: 0.3811 - acc: 0.8858 - val_loss: 0.4092 - val_acc: 0.8777

Epoch 00007: _f1_monitor improved from 0.44312 to 0.44322, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_6.h5
Epoch 8/100
105000/105000 [==============================] - 10s 100us/step - loss: 0.3709 - acc: 0.8887 - val_loss: 0.4018 - val_acc: 0.8794

Epoch 00008: _f1_monitor improved from 0.44322 to 0.46129, s


Epoch 00008: _f1_monitor did not improve from 0.59111
Epoch 9/100
105000/105000 [==============================] - 11s 102us/step - loss: 0.7027 - acc: 0.7313 - val_loss: 0.8112 - val_acc: 0.6952

Epoch 00009: _f1_monitor improved from 0.59111 to 0.61356, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_7.h5
Epoch 10/100
105000/105000 [==============================] - 11s 100us/step - loss: 0.6872 - acc: 0.7381 - val_loss: 0.8012 - val_acc: 0.6895

Epoch 00010: _f1_monitor did not improve from 0.61356
Epoch 11/100
105000/105000 [==============================] - 11s 102us/step - loss: 0.6741 - acc: 0.7439 - val_loss: 0.7969 - val_acc: 0.6939

Epoch 00011: _f1_monitor did not improve from 0.61356
Epoch 12/100
105000/105000 [==============================] - 11s 109us/step - loss: 0.6614 - acc: 0.7501 - val_loss: 0.8043 - val_acc: 0.6945

Epoch 00012: _f1_monitor did not improve from 0.61356
Epoch 13/100
105000/105000 [=================

105000/105000 [==============================] - 11s 100us/step - loss: 0.3762 - acc: 0.8728 - val_loss: 0.4714 - val_acc: 0.8428

Epoch 00023: _f1_monitor did not improve from 0.51087

Epoch 00023: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
The 8th grain: f1_score - 0.5108706832256884, acc - 0.8447333333333333
Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 10s 96us/step - loss: 0.9534 - acc: 0.6174 - val_loss: 0.8645 - val_acc: 0.6466

Epoch 00001: _f1_monitor improved from -inf to 0.26815, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_9.h5
Epoch 2/100
105000/105000 [==============================] - 10s 99us/step - loss: 0.7725 - acc: 0.6882 - val_loss: 0.7132 - val_acc: 0.7122

Epoch 00002: _f1_monitor improved from 0.26815 to 0.38913, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_9.h5
Epoc


Epoch 00013: _f1_monitor improved from 0.49309 to 0.50165, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_10.h5

Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 14/100
105000/105000 [==============================] - 11s 103us/step - loss: 0.4962 - acc: 0.8218 - val_loss: 0.5934 - val_acc: 0.7909

Epoch 00014: _f1_monitor improved from 0.50165 to 0.51549, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_10.h5
Epoch 15/100
105000/105000 [==============================] - 11s 103us/step - loss: 0.4950 - acc: 0.8225 - val_loss: 0.5939 - val_acc: 0.7913

Epoch 00015: _f1_monitor did not improve from 0.51549
Epoch 16/100
105000/105000 [==============================] - 11s 101us/step - loss: 0.4939 - acc: 0.8234 - val_loss: 0.5940 - val_acc: 0.7906

Epoch 00016: _f1_monitor did not improve from 0.51549

Epoch 00016: ReduceLROnPlateau reducing learning

105000/105000 [==============================] - 11s 106us/step - loss: 0.7723 - acc: 0.6929 - val_loss: 0.7727 - val_acc: 0.6961

Epoch 00004: _f1_monitor improved from 0.32694 to 0.36284, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_12.h5
Epoch 5/100
105000/105000 [==============================] - 11s 104us/step - loss: 0.7375 - acc: 0.7084 - val_loss: 0.7648 - val_acc: 0.7083

Epoch 00005: _f1_monitor improved from 0.36284 to 0.40648, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_12.h5
Epoch 6/100
105000/105000 [==============================] - 11s 103us/step - loss: 0.7138 - acc: 0.7190 - val_loss: 0.7679 - val_acc: 0.7145

Epoch 00006: _f1_monitor improved from 0.40648 to 0.43875, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/fasttext/fasttext_12.h5
Epoch 7/100
105000/105000 [==============================] - 11s 102us/step - loss: 0.6970 

In [ ]:
MODEL_PRE = os.path.join(SAVED_PATH, MODEL_PRE)
# comps_1 = [(LinXiSingleModel, {'model_file': MODEL_PRE + str(i) + '.h5'}) for i in range(len(y_cols))]
# comp_model_1 = LinXiCompositeModel(comps_1)

In [ ]:
weights_files = [MODEL_PRE + str(i) + '.h5' for i in range(len(y_cols))]
comp_model.load_weights(weights_files)

In [ ]:
val_preds = comp_model.predict(val_x)
test_preds = comp_model.predict(test_x)

In [ ]:
f1s = 0
for i, (vy, vp) in enumerate(zip(val_y.values.T + 2, val_preds)):
    f1 = f1_score(vy, vp, average='macro')
    print("The %sth f1: %s" % (i, f1))
    f1s += f1
    
print("The average f1 of val is %s" % (f1s / len(y_cols)))

In [ ]:
res = test.copy()
for index, col in enumerate(LABELS):
    res[col] = test_preds[index] - 2

res.to_csv(RESULT_CSV, index = False)